# A modular GCD algorithm: Functions

The individual steps are the same as in the script. Given are two polynomials $a, b \in \mathbb{Z}[x_1, \dots, x_s][x_n]$ with $0 \leq s < n$. This notebook serves as the implementation of a unified function that calculates the GCD.

In [26]:
# Note: This function is a hack that uses GCD.
# Implement pp() another way?

def pp(poly, var):
    """
    Calculates the primitive part of a polynomial
    in a given variable.
    
    Parameters
    ----------
    poly: polynomial
        Polynomial.
    var: variable
        Variable to take the primitive part of.
        
    Returns
    -------
    pp: polynomial
        Primitive part.
    """
    return poly / gcd(poly.polynomial(var).list())

def cont(poly, var):
    return gcd(poly.polynomial(var).list())

In [27]:
import sympy

def interpolate(rpolys, I, var):
    """
    Interpolates a set of polynomials.
    
    Parameters
    ----------
    rpolys: array-like
        Array consisting of tuples `(r,poly)`, where `r`
        is an integer.
    I: MPolynomialRing
        Polynomial ring.
    var: variable
        The variable to interpolate.
        
    Returns
    -------
    Interpolated polynomial.
    """
    symb = sympy.Symbol(var.__repr__())
    
    rpolys_sympified = [(gr[0], sympy.sympify(gr[1].__repr__())) for gr in rpolys]
    
    result = sympy.polys.polyfuncs.interpolate(rpolys_sympified, symb)
    
    return I(result)

In [28]:
class r_int():
    """"
    Class to store the parameter `r`.
    
    Attributes
    ----------
    r : int
        The variable.
    r_max: int
        Maximum possible `r`. Used to prevent an
        infinite while-loop.

    Methods
    -------
    reset_r()
        Resets `r` to 0.
    get_new_r(a_prime, b_prime, xs, xn)
        Given two polynomials, get a new `r`.
    """
    def __init__(self, r_max=10000):
        self.r = 0
        self.r_max = r_max

    def reset_r(self):
        self.r = 0

    def get_new_r(self, a_prime, b_prime, xs, xn):
        while self.r < self.r_max:
            self.r += 1
            if a_prime.substitute({xs:self.r}).degree(xn) == a_prime.degree(xn):
                return self.r
            if b_prime.substitute({xs:self.r}).degree(xn) == b_prime.degree(xn):
                return self.r
        raise Exception("r exceeded r_max")

In [29]:
import pdb

In [30]:
def GCD_MODm(a, b, I):
    """
    Calculates the GCD of two multivariate polynomials
    using a modular algorithm.
    
    Parameters
    ----------
    a: polynomial
        First polynomial.
    b: polynomial
        Second polynomial.
    I: MPolynomialRing
        Polynomial ring.
        
    Returns
    -------
    gcd: polynomial
        GCD of the two polynomials.
    """
    # Checking if a or b are integers. Maybe there's a better
    # method, but this works for now
    # TODO: call own integer gcd algorithm
    if hasattr(a, "sqrt") or hasattr(b, "sqrt"):
        return a.gcd(b)
    
    # To avoid problems later
    assert a.args() == b.args()
    
    if a.is_univariate() or b.is_univariate():
        # TODO: Call own GCD_MOD
        return a.gcd(b)
    
    xn = a.args()[0]
    xs = a.args()[1]
    
    M = 1 + min(a.degree(xs), b.degree(xs))
    
    a_prime = pp(a, xn)
    b_prime = pp(b, xn)
    
    I_prime = I.remove_var(I.gens()[0])
    f = GCD_MODm(a.content(), b.content(), I_prime)
    
    d = GCD_MODm(a_prime.polynomial(xn).lc(), b_prime.polynomial(xn).lc(), I_prime)
    
    variable_r = r_int()
    
    while True:
        variable_r.get_new_r(a_prime, b_prime, xs, xn)
        
        g_prime = a_prime.substitute({xs:variable_r.r}).gcd(b_prime.substitute({xs:variable_r.r}))
        
        c = g_prime.polynomial(xn).lc()
        
        # In case the division fails
        if (d.substitute({xs:variable_r.r}) * g_prime).quo_rem(c)[1] != 0:
            continue
        
        g_r = (d.substitute({xs:variable_r.r}) * g_prime) / c
        
        while True:
            # This array will get more elements later
            gs = [(variable_r.r, g_r)]

            m = 1

            bad_degree = False
            while m <= M:
                variable_r.get_new_r(a_prime, b_prime, xs, xn)

                g_prime = a_prime.substitute({xs:variable_r.r}).gcd(b_prime.substitute({xs:variable_r.r}))

                c = g_prime.polynomial(xn).lc()

                # In case the division fails
                if (d.substitute({xs:variable_r.r}) * g_prime).quo_rem(c)[1] != 0:
                    continue

                g_r = (d.substitute({xs:variable_r.r}) * g_prime) / c

                if g_r.degree(xn) < gs[0][1].degree(xn):
                    break
                    bad_degree = True
                
                if g_r.degree(xn) == gs[0][1].degree(xn):
                    gs.append((variable_r.r, g_r))
                    
                    interim_result = interpolate(gs, I, xs)
                    
                    # Already testing if it's the GCD
                    if a.quo_rem(interim_result)[1] == 0 and b.quo_rem(interim_result)[1] == 0:
                        break
                    
                    m += 1

            if bad_degree == False:
                g_final = f * pp(interim_result, xn)
                if a.quo_rem(g_final)[1] == 0 and b.quo_rem(g_final)[1] == 0:
                    return g_final

In [31]:
I.<y,x> = PolynomialRing(QQ, order='lex')

a = 2*x^2*y^3 - x*y^3 + x^3*y^2 + 2*x^4 * y-x^3*y - 6*x*y + 3*y + x^5 - 3*x^2
b = 2*x*y^3 - y^3 - x^2*y^2 + x*y^2 - x^3*y + 4*x*y - 2*y + 2*x^2

In [32]:
GCD_MODm(a, b, I)

2*y*x - y + x^2

In [33]:
a.gcd(b)

2*y*x - y + x^2